In [11]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('../_resources/names.txt', 'r').read().splitlines()

In [3]:
chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}
itos = dict(sorted(itos.items(), key=lambda item: 0 if item[1] == '.' else 1))

In [65]:
N = torch.zeros((27, 27), dtype=torch.int32) # a tensor with shape (27,27)

In [78]:
xs, ys = [], []

for w in words:
# for w in words[:1]:
    chs = ['.'] + ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = [stoi[ch1], stoi[ch2]]
        ix2 = stoi[ch3]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

num = len(xs)

In [8]:
(xs, ys)

(tensor([[ 0,  0],
         [ 0,  5],
         [ 5, 13],
         [13, 13],
         [13,  1]]),
 tensor([ 5, 13, 13,  1,  0]))

In [79]:
xenc = F.one_hot(xs, num_classes=27).float()
xenc_cat = xenc.view(xenc.shape[0], -1)
# xenc.shape # torch.Size([5, 2, 27])
xenc_cat.shape

torch.Size([228146, 54])

In [91]:
g = torch.Generator().manual_seed(2147483647)

W = torch.randn((54, 27), generator=g, requires_grad=True) # normal dist, column vector, [27,1]

for k in range(10000):
    logits = xenc_cat @ W
    counts = logits.exp()

    #softmax
    probs = counts / counts.sum(1, keepdims=True)

    # probs[torch.arange(num), ys]  => likelihood
    # .log()                        => log-likelihood
    # - ...	                        => negative log-likelihood (NLL)
    # .mean()                       => mean NLL over batch
    loss = -probs[torch.arange(num), ys].log().mean()

    W.grad = None
    loss.backward()
    
    W.data -= 1.0 * W.grad

    if (k % 100 == 0):
        print(loss)

tensor(4.2325, grad_fn=<NegBackward0>)
tensor(3.1502, grad_fn=<NegBackward0>)
tensor(2.8669, grad_fn=<NegBackward0>)
tensor(2.7393, grad_fn=<NegBackward0>)
tensor(2.6613, grad_fn=<NegBackward0>)
tensor(2.6074, grad_fn=<NegBackward0>)
tensor(2.5680, grad_fn=<NegBackward0>)
tensor(2.5382, grad_fn=<NegBackward0>)
tensor(2.5151, grad_fn=<NegBackward0>)
tensor(2.4966, grad_fn=<NegBackward0>)
tensor(2.4815, grad_fn=<NegBackward0>)
tensor(2.4689, grad_fn=<NegBackward0>)
tensor(2.4582, grad_fn=<NegBackward0>)
tensor(2.4490, grad_fn=<NegBackward0>)
tensor(2.4410, grad_fn=<NegBackward0>)
tensor(2.4340, grad_fn=<NegBackward0>)
tensor(2.4278, grad_fn=<NegBackward0>)
tensor(2.4223, grad_fn=<NegBackward0>)
tensor(2.4173, grad_fn=<NegBackward0>)
tensor(2.4129, grad_fn=<NegBackward0>)
tensor(2.4089, grad_fn=<NegBackward0>)
tensor(2.4053, grad_fn=<NegBackward0>)
tensor(2.4020, grad_fn=<NegBackward0>)
tensor(2.3989, grad_fn=<NegBackward0>)
tensor(2.3962, grad_fn=<NegBackward0>)
tensor(2.3936, grad_fn=<N

In [102]:
out = []
g_offset = 0
g = torch.Generator().manual_seed(2147483647 + g_offset)

for i in range(100):
    chs = ['.', '.']
    while True:
        ix1, ix2 = stoi[chs[-2]], stoi[chs[-1]]

        # ---- NEW: neural network prediction
        x = torch.zeros((1, 54))  # 1 sample, 54 features
        x[0, ix1] = 1.0
        x[0, 27 + ix2] = 1.0

        logits = x @ W  # shape [1, 27]
        probs = F.softmax(logits, dim=1)  # shape [1, 27]
        ix3 = torch.multinomial(probs, num_samples=1, generator=g).item()
        # ----

        ch3 = itos[ix3]
        if ch3 == '.':
            out.append(''.join(chs[2:]))
            break
        chs.append(ch3)

print(out[:10])

['cexze', 'morlyurailaziaydamellimittain', 'lusan', 'ka', 'da', 'samiyaubrtthrigotai', 'morielliaugie', 'teda', 'kaleyla', 'sade']
